In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import math

In [ ]:
dataset = '151507'
input_dir = 'D:/dataset/'  # Replace it with your file path

adata = sc.read_visium(input_dir + dataset)
adata.var_names_make_unique()
adata.obs_names_make_unique()
adata_label = pd.read_csv(input_dir + dataset + '/' + dataset + '_truth.csv', index_col=0)
adata.obs['batch'] = adata_label['batch'].astype("category")
adata.obs['ground.truth'] = adata_label['ground.truth'].astype("category")
print(adata)
adata = adata[adata.obs['ground.truth'] != 'na']
print(adata)

In [ ]:
dataSet = adata.obs['ground.truth'].astype('category')
print(dataSet)
print(len(adata.obs['ground.truth']))

In [ ]:
sc.pp.neighbors(adata, use_rep="spatial")
adata

In [ ]:
adata.obsp['connectivities']

In [ ]:
matrix = adata.obsp['connectivities'].toarray()
print(np.array_equal(matrix, matrix.T))

neighbors = dict()
val = dict()
for i in range(0, adata.n_obs):
    c = list()
    d = list()
    for j in range(0, adata.n_obs):
        if matrix[i][j] != 0:
            c.append(j)
            d.append(matrix[i][j])
    neighbors[i] = c
    val[i] = d
    
del matrix

In [ ]:
from math import log

# 函数说明:计算给定数据集的经验熵(香农熵)
def calcShannonEnt(neighbors):
    numEntires = 0                                                     #数据集的行数
    labelCounts = {}                                                   #保存每个标签(Label)出现次数的字典
    for key in neighbors:                                              #对所有spot进行统计
        currentLabel = adata.obs['ground.truth'][key]                  #提取当前标签(Label)信息
        for key2 in neighbors[key]:                                    #对当前spot的每个邻居进行统计
            nextLabel = adata.obs['ground.truth'][key2]                #提取邻居标签(Label)信息
            new_key = adata.obs['ground.truth'][key]  + '_' + adata.obs['ground.truth'][key2]
            if new_key not in labelCounts.keys():                      #如果标签(Label)没有放入统计次数的字典,添加进去
                labelCounts[new_key] = 0
            labelCounts[new_key] += 1                                  #Label计数
            numEntires = numEntires + 1                                #边计数

    print(labelCounts)
    shannonEnt = 0.0                                                   #经验熵(香农熵)
    print(numEntires)
    for key in labelCounts:                                            #计算香农熵
        prob = float(labelCounts[key]) / numEntires                    #选择该标签(Label)的概率
        shannonEnt -= prob * log(prob, 2)                              #利用公式计算
    return shannonEnt                                                  #返回经验熵(香农熵)

calcShannonEnt(neighbors)